<a href="https://colab.research.google.com/github/ayush18ranjan/AI-DATASCIENCE/blob/main/AyushRanjan_sourcecode_zip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sqlite3
import pandas as pd
import openai  # Use Groq API
from sqlalchemy import create_engine

def load_dataset(csv_path, db_name="database.db"):
    """Load CSV dataset into an SQLite database."""
    df = pd.read_csv(csv_path)
    engine = create_engine(f'sqlite:///{db_name}')
    table_name = "data"
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Dataset loaded into table '{table_name}'")
    return db_name, table_name

def nl_to_sql(nl_query, schema):
    """Convert natural language to SQL query using Groq API."""
    prompt = f"""
    Convert the following natural language query into an SQL query:
    Schema: {schema}
    Query: {nl_query}
    """
    try:
        response = openai.ChatCompletion.create(
            model="mixtral-8x7b-32768",
            messages=[{"role": "system", "content": "You are a SQL query expert."},
                      {"role": "user", "content": prompt}]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Error generating SQL: {e}"

def correct_sql(sql_query):
    """Use Groq API to correct SQL syntax or logical errors."""
    prompt = f"Correct any errors in this SQL query: {sql_query}"
    try:
        response = openai.ChatCompletion.create(
            model="mixtral-8x7b-32768",
            messages=[{"role": "system", "content": "You are an SQL correction assistant."},
                      {"role": "user", "content": prompt}]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Error correcting SQL: {e}"

def execute_sql(db_name, sql_query):
    """Execute an SQL query and return results."""
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        conn.commit()
        conn.close()
        return results
    except sqlite3.Error as e:
        return f"SQL Execution Error: {e}"

def main():
    groq_api_key = input("Enter your Groq API key: ")
    openai.api_key = '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' # Set the API key globally

    dataset_path = input("Enter dataset CSV file path: ")
    db_name, table_name = load_dataset(dataset_path)

    while True:
        nl_query = input("Enter your natural language query (or type 'exit' to quit): ")
        if nl_query.lower() == 'exit':
            break

        schema_info = f"Table: {table_name}, Columns: {', '.join(pd.read_sql(f'SELECT * FROM {table_name} LIMIT 1', sqlite3.connect(db_name)).columns)}"
        sql_query = nl_to_sql(nl_query, schema_info)

        print("Generated SQL Query:", sql_query)

        corrected_sql = correct_sql(sql_query)
        print("Corrected SQL Query:", corrected_sql)

        results = execute_sql(db_name, corrected_sql)
        print("Query Results:", results)

if __name__ == "__main__":
    main()


Enter your Groq API key: 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
